In [5]:
base_model = "google/gemma-2-2b"
dataset_name = "gbharti/finance-alpaca"
prompt_format = '''<|begin_of_text|><|start_header_id|>User: <|end_header_id|> {}<|eot_id|><|start_header_id|>Assistant: <|end_header_id|> {}<|eot_id|>'''
test_prompt_format = '''<|begin_of_text|><|start_header_id|>User: <|end_header_id|> {}<|eot_id|><|start_header_id|>Assistant: <|end_header_id|>'''

In [6]:
!pip install datasets
!pip install peft evaluate trl bitsandbytes accelerate


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [7]:
import huggingface_hub

huggingface_hub.login("<HF_AUTH_TOKEN>")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [20]:
from datasets import load_dataset, Dataset
dataset = load_dataset(dataset_name, split="train[:10000]")

In [21]:
dataset

Dataset({
    features: ['text', 'instruction', 'input', 'output'],
    num_rows: 10000
})

In [22]:
def text_formatter(example):
  return {'text': prompt_format.format(example['instruction'], example['output'])}

dataset = dataset.map(text_formatter)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [23]:
from datasets import DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForCausalLM,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

from trl import SFTTrainer
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [38]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model, trust_remote_code=True, device_map='cuda:0')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [39]:
input_text = "in 20 words give me the advantages of investing in 401k"
test_prompt = test_prompt_format.format(input_text)

tokenized_test = tokenizer(test_prompt, return_tensors='pt').to(model.device)

test_output = model.generate(**tokenized_test, max_new_tokens=256)
print(tokenizer.decode(test_output[0]))

<bos><|begin_of_text|><|start_header_id|>User: <|end_header_id|> in 20 words give me the advantages of investing in 401k<|eot_id|><|start_header_id|>Assistant: <|end_header_id|>

<|begin_of_text|><|start_header_id|>User: <|end_header_id|>

<|begin_of_text|><|start_header_id|>Assistant: <|end_header_id|>

<|begin_of_text|><|start_header_id|>User: <|end_header_id|>

<|begin_of_text|><|start_header_id|>Assistant: <|end_header_id|>

<|begin_of_text|><|start_header_id|>User: <|end_header_id|>

<|begin_of_text|><|start_header_id|>Assistant: <|end_header_id|>

<|begin_of_text|><|start_header_id|>User: <|end_header_id|>

<|begin_of_text|><|start_header_id|>Assistant: <|end_header_id|>

<|begin_of_text|><|start_header_id|>User: <|end_header_id|>

<|begin_of_text|><|start_


In [13]:
!pip install wandb


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [14]:
import wandb

wandb.login()

wandb: Currently logged in as: csreddy1998 (csreddy1998-none). Use `wandb login --relogin` to force relogin


True

In [40]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
output_dir = "./finnancial_advisor_A005"
per_device_train_batch_size = 1
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_steps = 200
num_train_epochs = 1
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "linear"

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    target_modules=['down_proj','o_proj','k_proj','q_proj','gate_proj','up_proj','v_proj'],
)

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to="wandb"
)
model.config.use_cache = False

In [41]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=256,
    tokenizer=tokenizer,
    args=training_args,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs


In [42]:
trainer.train()

Step,Training Loss
1,2.220400
2,2.787900
3,2.625900
4,2.718800
5,2.388600
6,2.244100
7,2.565300
8,2.169500
9,1.971400
10,2.010000


TrainOutput(global_step=100, training_loss=1.891430263519287, metrics={'train_runtime': 54.3089, 'train_samples_per_second': 3.683, 'train_steps_per_second': 1.841, 'total_flos': 529353635839488.0, 'train_loss': 1.891430263519287, 'epoch': 0.02})

In [44]:
input_text = "give me the advantages of investing in 401k"
test_prompt = test_prompt_format.format(input_text)

tokenized_test = tokenizer(test_prompt, return_tensors='pt').to(model.device)

test_output = model.generate(**tokenized_test, max_new_tokens=256)
print(tokenizer.decode(test_output[0]))


<bos><|begin_of_text|><|start_header_id|>User: <|end_header_id|> give me the advantages of investing in 401k<|eot_id|><|start_header_id|>Assistant: <|end_header_id|> The main advantage is that you can invest in a 401k without having to pay taxes on the money you invest.  If you invest in a 401k, you can deduct the amount you invest from your taxable income.  This means that you can save money on taxes.  Another advantage is that you can invest in a 401k without having to pay taxes on the money you invest.  If you invest in a 401k, you can deduct the amount you invest from your taxable income.  This means that you can save money on taxes.  Another advantage is that you can invest in a 401k without having to pay taxes on the money you invest.  If you invest in a 401k, you can deduct the amount you invest from your taxable income.  This means that you can save money on taxes.  Another advantage is that you can invest in a 401k without having to pay taxes on the money you invest.  If you i